In [1]:
import os

import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.metrics import auc, confusion_matrix, roc_curve
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.metrics import AUC, BinaryAccuracy, Precision, Recall
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import src.graphics as graphics

In [2]:
VERSION = 1
MODEL_BASE_NAME = "MegaClassifier_a_EfficientNetB5"

DATASET_CSV = os.path.abspath(
    "./data/processed/onlyDetectionsForTrain/onlyDetectionsForTrain.csv"
)
DATASET_PATH = os.path.dirname(DATASET_CSV)

In [3]:
dataset = pd.read_csv(DATASET_CSV, sep=";")
dataset["file_name"] = dataset["file_name"].apply(
    lambda x: os.path.join(DATASET_PATH, x)
)
dataset["binary_label"] = dataset["binary_label"].astype(str)

train_dataset = dataset[dataset["subset"] == "train"]
validationtrain_dataset = dataset[dataset["subset"] == "validation"]
test_dataset = dataset[dataset["subset"] == "test"]

EPOCHS = 10
IMAGE_SIZE = (456, 456)
IMAGE_SHAPE = IMAGE_SIZE + (3,)
SEED = 42

In [4]:
OPTIMIZERS = {"Adam": tf.keras.optimizers.legacy.Adam()}

In [5]:
LOSS = {"BinaryCrossentropy": tf.keras.losses.BinaryCrossentropy()}

In [6]:
METRICS = [
    BinaryAccuracy(name="accuracy"),
    Precision(name="precision"),
    Recall(name="recall"),
    AUC(name="auc"),
]

2025-03-02 11:48:53.689263: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M3
2025-03-02 11:48:53.689312: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2025-03-02 11:48:53.689332: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2025-03-02 11:48:53.692057: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:303] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-03-02 11:48:53.692119: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:269] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [7]:
BATCH_SIZES = [8, 16, 32, 64, 128]

In [8]:
SUBVERSION = 0
for BATCH_SIZE in BATCH_SIZES:
    LOGS_PATH = os.path.abspath(
        f"./logs/{MODEL_BASE_NAME}/v{VERSION}/v{VERSION}.{SUBVERSION}"
    )
    MODELS_PATH = os.path.abspath(
        f"./models/{MODEL_BASE_NAME}/v{VERSION}/v{VERSION}.{SUBVERSION}"
    )
    REPORTS_PATH = os.path.abspath(
        f"./reports/{MODEL_BASE_NAME}/v{VERSION}/v{VERSION}.{SUBVERSION}"
    )

    MODEL_COMPLETE_NAME = f"{MODEL_BASE_NAME} v{VERSION}.{SUBVERSION}"

    train_datagen = ImageDataGenerator(
        preprocessing_function=tf.keras.applications.efficientnet.preprocess_input,
    )
    train_images = train_datagen.flow_from_dataframe(
        dataframe=train_dataset,
        x_col="file_name",
        y_col="binary_label",
        target_size=IMAGE_SIZE,
        batch_size=BATCH_SIZE,
        class_mode="binary",
        shuffle=True,
        seed=SEED,
    )

    datagen = ImageDataGenerator(
        preprocessing_function=tf.keras.applications.efficientnet.preprocess_input,
    )
    validation_images = datagen.flow_from_dataframe(
        dataframe=validationtrain_dataset,
        x_col="file_name",
        y_col="binary_label",
        target_size=IMAGE_SIZE,
        batch_size=BATCH_SIZE,
        class_mode="binary",
        shuffle=True,
        seed=SEED,
    )
    test_images = datagen.flow_from_dataframe(
        dataframe=test_dataset,
        x_col="file_name",
        y_col="binary_label",
        target_size=IMAGE_SIZE,
        batch_size=BATCH_SIZE,
        class_mode="binary",
        shuffle=False,
        seed=SEED,
    )

    pretrained_model = tf.keras.applications.EfficientNetB5(
        weights="imagenet",
        include_top=False,
        input_shape=IMAGE_SHAPE,
    )
    pretrained_model.trainable = False

    model = tf.keras.Sequential(
        [
            pretrained_model,
            tf.keras.layers.GlobalAveragePooling2D(),
            tf.keras.layers.Dense(1, activation="sigmoid"),
        ],
        name=f"{MODEL_BASE_NAME}_v{VERSION}.{SUBVERSION}",
    )

    model.compile(
        optimizer=OPTIMIZERS["Adam"],
        loss=LOSS["BinaryCrossentropy"],
        metrics=METRICS,
    )

    history = model.fit(
        train_images,
        epochs=EPOCHS,
        validation_data=validation_images,
        callbacks=[TensorBoard(log_dir=LOGS_PATH)],
    )

    dataframe = pd.DataFrame(history.history)
    history_path = os.path.join(LOGS_PATH, f"history_v{VERSION}.{SUBVERSION}.csv")
    dataframe.to_csv(history_path, sep=";", index=False)

    os.makedirs(REPORTS_PATH, exist_ok=True)
    accuracy_chart = graphics.create_training_accuracy_chart(
        history_path=history_path,
        model_name=MODEL_COMPLETE_NAME,
    )
    accuracy_chart.write_image(f"{REPORTS_PATH}/accuracy_v{VERSION}.{SUBVERSION}.png")

    loss_chart = graphics.create_training_loss_chart(
        history_path=history_path,
        model_name=MODEL_COMPLETE_NAME,
    )
    loss_chart.write_image(f"{REPORTS_PATH}/loss_v{VERSION}.{SUBVERSION}.png")

    os.makedirs(MODELS_PATH, exist_ok=True)
    model.save(MODELS_PATH)

    results = model.evaluate(test_images)
    metric_names = history.model.metrics_names
    evaluation_results = {
        ("test_" + name): value for name, value in zip(metric_names, results)
    }

    evaluation = pd.DataFrame([evaluation_results])
    evaluation.to_csv(
        os.path.join(LOGS_PATH, f"evaluation_v{VERSION}.{SUBVERSION}.csv"),
        sep=";",
        index=False,
    )

    y_pred_prob = model.predict(test_images)
    y_true = test_images.labels

    fpr, tpr, thresholds = roc_curve(y_true, y_pred_prob)
    roc_auc = auc(fpr, tpr)

    roc_curve_chart = graphics.create_roc_curve_chart(
        fpr=fpr,
        tpr=tpr,
        roc_auc=roc_auc,
        model_name=MODEL_COMPLETE_NAME,
    )
    roc_curve_chart.write_image(f"{REPORTS_PATH}/roc_curve_v{VERSION}.{SUBVERSION}.png")

    optimal_idx = np.argmax(tpr - fpr)
    optimal_threshold = thresholds[optimal_idx]

    # y_pred_prob = y_pred_prob.flatten()
    y_pred_class = (y_pred_prob > optimal_threshold).astype(int)
    conf_matrix = confusion_matrix(y_true, y_pred_class)
    conf_matrix_text = [[str(value) for value in row] for row in conf_matrix]

    # Crear el gráfico correctamente
    confusion_matrix_chart = graphics.create_confusion_matrix_chart(
        conf_matrix=conf_matrix,
        conf_matrix_text=conf_matrix_text,
        model_name=MODEL_COMPLETE_NAME,
    )
    confusion_matrix_chart.write_image(
        f"{REPORTS_PATH}/confusion_matrix_optimal_v{VERSION}.{SUBVERSION}.png"
    )

    y_pred_class = (y_pred_prob > 0.5).astype(int)
    conf_matrix = confusion_matrix(y_true, y_pred_class)
    conf_matrix_text = [[str(value) for value in row] for row in conf_matrix]

    confusion_matrix_chart = graphics.create_confusion_matrix_chart(
        conf_matrix=conf_matrix,
        conf_matrix_text=conf_matrix_text,
        model_name=MODEL_COMPLETE_NAME,
    )
    confusion_matrix_chart.write_image(
        f"{REPORTS_PATH}/confusion_matrix_v{VERSION}.{SUBVERSION}.png"
    )

    SUBVERSION += 1
    print("\n\n")


Found 17906 validated image filenames belonging to 2 classes.
Found 4286 validated image filenames belonging to 2 classes.
Found 4286 validated image filenames belonging to 2 classes.
115263384/115263384 [==============================] - 3s 0us/step
Epoch 1/10


2025-03-02 11:49:04.732558: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


2239/2239 [==============================] - ETA: 0s - loss: 0.2272 - accuracy: 0.9066 - precision: 0.9289 - recall: 0.9460 - auc: 0.9613

2025-03-02 12:08:31.437542: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


2239/2239 [==============================] - 1442s 641ms/step - loss: 0.2272 - accuracy: 0.9066 - precision: 0.9289 - recall: 0.9460 - auc: 0.9613 - val_loss: 0.1661 - val_accuracy: 0.9386 - val_precision: 0.9475 - val_recall: 0.9605 - val_auc: 0.9828
Epoch 2/10
2239/2239 [==============================] - 1414s 632ms/step - loss: 0.1744 - accuracy: 0.9267 - precision: 0.9478 - recall: 0.9534 - auc: 0.9767 - val_loss: 0.1476 - val_accuracy: 0.9419 - val_precision: 0.9615 - val_recall: 0.9503 - val_auc: 0.9848
Epoch 3/10
2239/2239 [==============================] - 1397s 624ms/step - loss: 0.1616 - accuracy: 0.9319 - precision: 0.9526 - recall: 0.9554 - auc: 0.9799 - val_loss: 0.1445 - val_accuracy: 0.9477 - val_precision: 0.9638 - val_recall: 0.9570 - val_auc: 0.9854
Epoch 4/10
2239/2239 [==============================] - 1396s 623ms/step - loss: 0.1555 - accuracy: 0.9335 - precision: 0.9541 - recall: 0.9559 - auc: 0.9813 - val_loss: 0.1418 - val_accuracy: 0.9473 - val_precision: 0.959

...

140/140 [==============================] - 1588s 11s/step - loss: 0.5847 - accuracy: 0.7762 - precision: 0.7705 - recall: 0.9840 - auc: 0.6734 - val_loss: 0.6542 - val_accuracy: 0.6617 - val_precision: 0.6618 - val_recall: 0.9993 - val_auc: 0.5085
Epoch 2/10
140/140 [==============================] - 1571s 11s/step - loss: 0.5792 - accuracy: 0.7387 - precision: 0.7388 - recall: 0.9997 - auc: 0.4971 - val_loss: 0.6484 - val_accuracy: 0.6624 - val_precision: 0.6624 - val_recall: 0.9989 - val_auc: 0.5105
Epoch 3/10
140/140 [==============================] - 1574s 11s/step - loss: 0.5777 - accuracy: 0.7388 - precision: 0.7391 - recall: 0.9992 - auc: 0.5057 - val_loss: 0.6525 - val_accuracy: 0.6622 - val_precision: 0.6622 - val_recall: 0.9989 - val_auc: 0.5065
Epoch 4/10
140/140 [==============================] - 1575s 11s/step - loss: 0.5803 - accuracy: 0.7392 - precision: 0.7392 - recall: 0.9996 - auc: 0.4976 - val_loss: 0.6868 - val_accuracy: 0.6612 - val_precision: 0.6616 - val_recall: 

INFO:tensorflow:Assets written to: /Users/litto/WORKSPACE/MegaClassifier/models/MegaClassifier_a_EfficientNetB5/v1/v1.4/assets


34/34 [==============================] - 300s 9s/step - loss: 0.6440 - accuracy: 0.6610 - precision: 0.6618 - recall: 0.9975 - auc: 0.5086


2025-03-03 09:00:03.437238: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


34/34 [==============================] - 300s 9s/step



